# 02. Understand Omok

Based on
https://github.com/lumiknit/mock5.py
and
https://github.com/lumiknit/mock5-q



## 오목

오목은 본질적으로는 틱택토와 크게 다르지 않습니다.
몇가지 차이점을 짚어보자면,

- 한 줄에 3개 대신 5개의 돌을 두어야 승리합니다.
- 판이 승리 조건에 비해 **매우** 큽니다.
틱택토에서는 가로세로 3으로, 판 크기에 딱 맞게 돌을 채워야 승리 조건이 만족하지만, 오목은 보통 가로세로 15로
승리 조건을 3번 채울 수 있을 정도입니다.
- 이 게임 역시 흑이 압도적으로 유리하다고 알려져 있기 때문에, 돌을 놓지 못하는 특수한 조건이 존재합니다. (장목, 삼삼, 사사 등)

### 그냥 틱택토처럼 학습시키면 안 될까?

하지만 어쨌든 판 크기가 늘어났다는 점 외에는 틱택토와 게임의 진행이 같기 때문에, 틱택토에서 했던 것처럼 DQN을 만들어서 학습해보는 것을 생각해볼 수 있습니다만, 실제로는 잘 안 됩니다.

https://github.com/lumiknit/mock5-q

가장 큰 문제점은 오목은 상태와 행동의 공간이
너무 크다는 것입니다.
상태는 최대 $3^{225} \simeq 2^{356} \simeq 10^{107}$가지로, 너무 많으며,
행동도 $225$ 가지로 절대 적은 수가 아닙니다.
따라서 각 상태에 대해 적절한 행동을 학습시키는 것은
매우 어렵습니다.

따라서 오목판을 통째로 입력받는 대신에
적당히 추려서 적절한 입력으로 축소를 시킬 필요가 있습니다.

## 오목 게임

여기서는 lumiknit/mock5.py를 사용합니다.
(`lumiknit/mock5.py/mock5/__init__.py`)

아래는 minified code이므로, 원본은 github에서 확인해주시면 됩니다.

Class 자체는 이전의 TicTacToe와 크게 다를 것이 없습니다.

In [ ]:
_C=False
_B=True
_A=None
EMPTY=0
BLACK=1
WHITE=2
_STONE_CHAR=['.','O','X']
def _digit_to_int(s,offset=0):
	v=ord(s[offset])
	if 48<=v and v<48+10:return v-48
	elif 65<=v and v<65+26:return v-65+10
	elif 97<=v and v<97+26:return v-97+10
	else:return _A
def _int_to_digit(v):
	if 0<=v and v<10:return chr(48+v)
	elif 10<=v and v<10+26:return chr(65+v-10)
	else:return _A
class Mock5:
	def __init__(self,height=15,width=15,board=_A,history=_A):
		if type(height)is not int or type(width)is not int:raise TypeError
		if height<=0 or height>36 or width<=0 or width>36:raise Exception('Mock5 board size should be between 1 and 36!')
		self.height=height;self.width=width
		if board is not _A:
			if len(board)!=height*width:raise ValueError
			self.board=list(board)
		else:self.board=[0]*(self.height*self.width)
		self.player=1;self.history=[]
		if history is not _A:
			for idx in history:r,c=self._expand_index(idx);self.place_stone(r,c)
	def __str__(self):
		A=' {}';r='=====================================';r+="\n [ Turn {:3d} ; {}P's turn (tone = {}) ]".format(len(self.history),self.player,_STONE_CHAR[self.player]);r+='\n  |'
		for i in range(self.width):r+=A.format(_int_to_digit(i))
		r+='\n--+'
		for i in range(self.width):r+='--'
		for i in range(self.height):
			r+='\n{} |'.format(_int_to_digit(i))
			for j in range(self.width):r+=A.format(_STONE_CHAR[self.board[i*self.width+j]])
		return r
	def _reduce_index(self,r,c):return r*self.width+c
	def _expand_index(self,idx):return idx//self.width,idx%self.width
	def _check_key(self,key):
		if type(key)is not tuple or len(key)!=2:raise TypeError
		if type(key[0])is not int or type(key[1])is not int:raise TypeError
		if key[0]<0 or key[0]>=self.height:raise IndexError
		if key[1]<0 or key[1]>=self.width:raise IndexError
	def __getitem__(self,key):self._check_key(key);return self.board[self._reduce_index(key[0],key[1])]
	def __setitem__(self,key,value):
		self._check_key(key)
		if type(value)is not int or value<0 or value>2:raise TypeError
		self.board[self._reduce_index(key[0],key[1])]=value
	def duplicate(self):return self.__class__(self.height,self.width,board=self.board)
	def replay(self):return self.__class__(self.height,self.width,history=self.history)
	def rotate_ccw(self):
		def rotate_idx(idx):r,c=self._expand_index(idx);c=self.width-c-1;return c*self.height+r
		new_history=list(map(rotate_idx,self.history));return self.__class__(self.width,self.height,history=new_history)
	def flip_vertical(self):
		def flip_idx(idx):r,c=self._expand_index(idx);r=self.height-r-1;return r*self.width+c
		new_history=list(map(flip_idx,self.history));return self.__class__(self.height,self.width,history=new_history)
	class _IndexIter:
		def __init__(self,game,sr,sc,dr,dc):
			if dr==0 and dc==0:raise ValueError
			self.game=game;self.r=sr;self.c=sc;self.dr=dr;self.dc=dc
			if dr>=0:self.br=self.game.height
			else:self.br=-1
			if dc>=0:self.bc=self.game.width
			else:self.bc=-1
		def __iter__(self):return self
		def __next__(self):
			if self.r==self.br or self.c==self.bc:raise StopIteration
			else:ret=self.r,self.c;self.r+=self.dr;self.c+=self.dc;return ret
	def first_of_row(self,idx):
		if idx<0 or idx>=self.height:raise IndexError
		return idx,0
	def first_of_column(self,idx):
		if idx<0 or idx>=self.width:raise IndexError
		return 0,idx
	def first_of_right_down(self,idx):
		r,c=0,0
		if idx<0:raise IndexError
		elif idx<self.height:r=self.height-idx-1
		elif idx<self.width+self.height-1:c=idx-self.height+1
		else:raise IndexError
		return r,c
	def first_of_left_down(self,idx):
		r,c=0,self.width-1
		if idx<0:raise IndexError
		elif idx<self.width:c=idx
		elif idx<self.width+self.height-1:r=idx-self.width+1
		else:raise IndexError
		return r,c
	def iter_row(self,idx):r,c=self.first_of_row(idx);return self._IndexIter(self,r,c,0,1)
	def iter_column(self,idx):r,c=self.first_of_column(idx);return self._IndexIter(self,r,c,1,0)
	def iter_right_down(self,idx):r,c=self.first_of_right_down(idx);return self._IndexIter(self,r,c,1,1)
	def iter_left_down(self,idx):r,c=self.first_of_left_down(idx);return self._IndexIter(self,r,c,1,-1)
	def slice_row(self,idx):return[self[(r,c)]for(r,c)in self.iter_row(idx)]
	def slice_column(self,idx):return[self[(r,c)]for(r,c)in self.iter_column(idx)]
	def slice_right_down(self,idx):return[self[(r,c)]for(r,c)in self.iter_right_down(idx)]
	def slice_left_down(self,idx):return[self[(r,c)]for(r,c)in self.iter_left_down(idx)]
	def can_place_at(self,r,c,player=_A):
		if type(r)is not int or type(c)is not int:raise TypeError
		if r<0 or r>=self.height or c<0 or c>=self.width:raise IndexError
		if player==_A:player=self.player
		is_empty=self[(r,c)]==0;return is_empty
	def place_stone(self,r,c,player=_A):
		if not self.can_place_at(r,c,player):return _C
		if player==_A:player=self.player;self.player=3-self.player;self.history.append(self._reduce_index(r,c))
		self[(r,c)]=player;return _B
	def place_stone_at_index(self,idx,player=_A):r,c=self._expand_index(idx);return self.place_stone(r,c,player)
	def history_depth(self):return len(self.history)
	def undo(self):
		if len(self.history)>0:
			idx=self.history.pop()
			if self.board[idx]!=3-self.player:raise Exception('Board is corrupted!')
			self.board[idx]=0;self.player=3-self.player
		return len(self.history)
	def _scan_with_iter(self,iter):
		cnt=1;p=self[iter.__next__()]
		for (r,c) in iter:
			cnt=cnt+1 if self[(r,c)]==p else 1;p=self[(r,c)]
			if cnt>=5 and p>0:return p
		return _A
	def check_win(self):
		for x in range(self.height):
			v=self._scan_with_iter(self.iter_row(x))
			if v is not _A:return v
		for x in range(self.width):
			v=self._scan_with_iter(self.iter_column(x))
			if v is not _A:return v
		for x in range(self.height+self.width-1):
			v=self._scan_with_iter(self.iter_right_down(x))
			if v is not _A:return v
			v=self._scan_with_iter(self.iter_left_down(x))
			if v is not _A:return v
		if len(self.history)>=self.width*self.height:return 0
		return _A
	def play(self,input1=_A,input2=_A,random_first=_B,print_intermediate_state=_B,print_messages=_B):
		def user_input(game):
			while _B:
				v=input("row-col (e.g. 3a, 77) ; 'gg' ; 'undo' > ").strip()
				try:
					if v=='gg':return _C,0
					elif v=='undo':return _C,1
					v=[y for x in map(list,v.split())for y in x]
					if len(v)!=2:raise Exception()
					r=_digit_to_int(v[0]);c=_digit_to_int(v[1])
					if r is _A or c is _A:raise Exception()
					if not self.can_place_at(r,c):raise IndexError()
					return r,c
				except IndexError:print('Cannot place stone at {}, {}!'.format(v[0],v[1]))
				except Exception:print('Wrong input!')
				finally:0
		user_input.name='user';import random;exchanged=_C
		if random_first and random.random()<0.5:exchanged=_B;input1,input2=input2,input1
		if input1 is _A:input1=user_input
		if input2 is _A:input2=user_input
		pif=[_A,input1,input2]
		def player_name(idx):
			A='{}p ({})'
			if hasattr(pif[idx],'name'):return A.format(idx,pif[idx].name)
			else:return A.format(idx,pif[idx])
		winner=_A
		while _B:
			if print_intermediate_state:print(str(self))
			ret=pif[self.player](self)
			if ret is _A:r,c=_C,0
			else:r,c=ret
			if r is _A or r is _C:
				if c==1:self.undo()
				elif c==0:
					if print_messages:print('{} give up!'.format(player_name(self.player)))
					winner=3-self.player;break
			elif not self.place_stone(r,c):
				if print_messages:print('{} cheats! (try to place stone at {}, {})'.format(player_name(self.player),r,c))
				winner=3-self.player;break
			winner=self.check_win()
			if winner!=_A:
				if print_messages:
					print(str(self))
					if winner==0:print('Draw!')
					else:print('{} win!'.format(player_name(winner)))
				break
		if winner is not _A and winner>0 and exchanged:winner=3-winner
		return winner
	def _map_for_player(self,player=_A):
		if player==_A:player=self.player
		return[0,player,3-player]
	def board_for(self,player=_A):m=self._map_for_player(player);return[m[x]for x in self.board]
	def one_hot_encoding(self,player=_A):
		m=self._map_for_player(player);a=[[0]*(self.height*self.width)for _ in range(3)]
		for i in range(self.height*self.width):a[m[self.board[i]]][i]=1
		return a
	def numpy(self,player=_A,one_hot_encoding=_B,rank=_A,dtype=_A):
		import numpy as np
		if dtype is _A:dtype=np.float
		if one_hot_encoding:
			a=self.one_hot_encoding(player=player);n=np.array(a,dtype=dtype)
			if rank==1:n=n.reshape(-1)
			elif rank==2:0
			else:n=n.reshape(3,self.height,self.width)
			return n
		else:
			a=self.board_for(player=player);n=np.array(a)
			if rank==2:n=n.reshape(self.width,self.height)
			return n
	def tensor(self,player=_A,one_hot_encoding=_B,rank=_A,dtype=_A):
		import torch
		if dtype is _A:dtype=torch.float
		if one_hot_encoding:
			a=self.one_hot_encoding(player=player);n=torch.tensor(a,dtype=dtype)
			if rank==1:n=n.view(-1)
			elif rank==2:0
			else:n=n.view(3,self.height,self.width)
			return n
		else:
			a=self.board_for(player=player);n=torch.tensor(a)
			if rank==2:n=n.view(self.width,self.height)
			return n
	def empty_array(self,empty=_B,non_empty=_C):m=[empty,non_empty,non_empty];return[m[self.board[idx]]for idx in range(self.height*self.width)]
	def empty_numpy(self,rank=1,empty=1.0,non_empty=0.0,dtype=_A):
		import numpy as np
		if dtype is _A:dtype=type(empty)
		em=self.empty_array(empty,non_empty);arr=np.array(em,dtype=dtype)
		if rank==2:return arr.reshape(self.height,self.width)
		else:return arr
	def empty_tensor(self,rank=1,empty=_B,non_empty=_C,dtype=_A):
		import torch
		if dtype is _A:dtype=type(empty)
		em=self.empty_array(empty,non_empty);tensor=torch.tensor(em,dtype=dtype)
		if rank==2:return tensor.view(self.height,self.width)
		return tensor

In [ ]:
# Mock5().play()

 [ Turn   0 ; 1P's turn (tone = O) ]
  | 0 1 2 3 4 5 6 7 8 9 A B C D E
--+------------------------------
0 | . . . . . . . . . . . . . . .
1 | . . . . . . . . . . . . . . .
2 | . . . . . . . . . . . . . . .
3 | . . . . . . . . . . . . . . .
4 | . . . . . . . . . . . . . . .
5 | . . . . . . . . . . . . . . .
6 | . . . . . . . . . . . . . . .
7 | . . . . . . . . . . . . . . .
8 | . . . . . . . . . . . . . . .
9 | . . . . . . . . . . . . . . .
A | . . . . . . . . . . . . . . .
B | . . . . . . . . . . . . . . .
C | . . . . . . . . . . . . . . .
D | . . . . . . . . . . . . . . .
E | . . . . . . . . . . . . . . .
row-col (e.g. 3a, 77) ; 'gg' ; 'undo' > 3a
 [ Turn   1 ; 2P's turn (tone = X) ]
  | 0 1 2 3 4 5 6 7 8 9 A B C D E
--+------------------------------
0 | . . . . . . . . . . . . . . .
1 | . . . . . . . . . . . . . . .
2 | . . . . . . . . . . . . . . .
3 | . . . . . . . . . . O . . . .
4 | . . . . . . . . . . . . . . .
5 | . . . . . . . . . . . . . . .
6 | . . . . . . . . . . . . . . .

2

다음과 같이 랜덤으로 행동하는 agent를 만들 수도 있습니다.

In [ ]:
def agent_random(game):
  import random
  idx = random.randint(0, game.height * game.width)
  for off in range(game.height * game.width):
    rdx = (idx + off) % (game.height * game.width)
    r, c = game._expand_index(rdx)
    if game.can_place_at(r, c):
      return (r, c)

In [ ]:
Mock5().play(agent_random, agent_random)

## 오목을 이해하자

오목 역시 둘 중 한 사람에게는 필승법이 존재하지만,
현 상황에서 어떤 수가 최선의 수인지 알기 힘듭니다.
실제로도 (천재적인 오목 플레이어도 그런지는 모르겠지만)
보통 사람은 몇 수 뒤에 반드시 이길 수 있는
몇몇 수를 제외한다면, 이렇게 두면 이길 확률이
좋을 것이라든가 모양이 괜찮다는 등의 추측을 바탕으로
수를 둡니다.
즉, 각 위치에 일정한 가치를 두고 고려를 하게 됩니다.

가치를 어떻게 매기느냐는 매우 어렵습니다.
예를 들어서 체스의 경우에는 폰 1개를 1점이라고 두고
다른 말들을 폰에 비교해서 얼마나 더 가치가 있는지를
판단하게 되며,
과거부터 계속 된 연구를 통해 기물에 어느 정도 가치가
있는지 정립된 것이 있습니다.

오목에서는 이렇게 가치를 평가할 말은 없지만,
현재 돌의 배치에서 몇 수 정도에 오목을 만들어낼 수
있는지를 고려해볼 수 있습니다.
예를 들어서 돌 개수로 아래와 같이 경우를 나눌 수 있습니다.

- 우선 5목을 만들기 위해서는 연속된 5개 칸에
상대의 돌이 있어서는 안 됩니다.
(즉, 자신의 돌과 빈칸만 있는 경우
5목을 만드는 것이 가능)
여기서 $n$목은 자신의 돌을 $(5-n)$개를 추가로 두었을 때
5목이 되는 경우를 말하도록 합시다.
- 자신의 5목이 있으면 이미 승부가 결정되어 있습니다.
- 연속으로 자신의 돌이 4개가 놓이고, 양 옆이 빈칸인 경우는 자신의 차례든 상대의 차례든 자신의 승리입니다. (열린 4목)
- 자신의 4목이 있으면 자신의 차례면 승리이며,
상대의 차례라면 무조건 막아야 합니다.
- 만약 1수를 더 두어서 열린 4목이 되는 3목은,
자신의 차례면 열린 4목을 만들며 게임이 끝나며,
상대의 차례면 반드시 막아야 합니다. (열린 3목)
- 그 외의 3목의 경우 2수를 두면 5목이 되며, 상대는 그 2수 중 최소 1수에서 최대 2수로 막는 것이 가능합니다.

위와 같은 분류로 판 전체를 분석한 뒤에는
일부 최선의 수나 승패를 알 수 있습니다.

- 나의 4목이 있으면, 빈칸을 채워서 승리
- 상대의 4목이 있으면 빈칸을 채워서 막습니다.
- 만약에 상대의 4목이 2개 있으면 패배
- 나의 열린 3목이 있으면 연장하여 공격
- 상대의 열린 3목이 있으면 방어

이를 정리하면,

- 만약 자신이 두면 이기는 수가 하나 존재하면,
그곳에 돌을 두어서 이길 수 있습니다.
- 만약 자신이 막지 않으면 지는 수가 하나 존재하면,
그곳에 돌을 두어서 막아야합니다.
- 만약 자신이 막지 않으면 지는 수가 둘 이상 존재하면,
자신은 졌습니다.

와 같이 반드시 두는 수를 정리할 수 있고,
그 외의 경우에는 나와 상대의 수를 비교해서
한 모양이 완성되기까지 몇 수가 필요한지,
방어에 몇 수가 필요한지,
그러한 모양이 몇개가 있는지에 따라
행동을 결정해야 합니다.



### 간단한 분석 및 인공지능

오목에서 돌을 연결할 수 있는 방향은 가로, 세로, 두
대각선으로 총 4개가 있습니다.
여기서 우리는 특정 위치에 돌을 두었을 때
각 방향으로 돌이 어떻게 연결되느냐를 찾아야 합니다.

이 때 몇가지 고려할 점이 있는데,

- 일단 5목이 되려면 5개의 연속된 칸만 잘라서 봤을 때
한가지 색의 돌만 있어야 하기 때문에, 각 5개칸마다
2가지 돌이 공존하는 경우는 무시해도 됩니다.
- 3목이나 4목에서는 '열림'이 방어가 가능한지 여부를
결정하기 때문에 매우 중요합니다. 이 때문에 5개칸 뿐만
아니라 가장자리의 2개 칸(또는 벽)에 대한 고려가
필요합니다.
- 모양의 남은 수의 차이는 그 모양의 수보다 절대적인 경우가 많습니다. 예를 들어서 열린 3목이 아무리 많아도 4목 하나에 이기지 못하는 등이 있습니다. 보통 4목에 가까울 수록 이런 경향이 생깁니다.
- 한 수로 인해 여러 방향으로 연결이 이루어지는 위치가 더 중요합니다. 즉, 특정 위치의 가치는 각 방향별 가치의 최댓값이 아니라 합이 되어야 합니다.

그 외에도 가치를 부여할 떄 흔히 알고 있는 직관을
추가해볼 수 있습니다.

- 가능하면 초반에 가운데에 돌을 두는 것이 좋습니다. 가장자리에 가까우면 돌을 연결하다가 판 가장자리에 막히는
경우가 생기기 때문입니다.

## 분석 알고리즘

아래에서는 위 판단을 바탕으로,
판을 읽어서 각 위치의 가치를 계산합니다.

`lumiknit/mock5.py/mock5/analysis.py`

In [ ]:
def _sign(x):
  if x > 0: return 1
  elif x < 0: return -1
  else: return 0

N_OVER_5 = 7
N_5 = 6
N_OPEN_4 = 5
N_OPEN_3 = 4
N_4 = 3
N_3 = 2
N_2 = 1

B_OVER_5 = 0b1000000
B_5 = 0b100000
B_OPEN_4 = 0b10000
B_OPEN_3 = 0b1000
B_4 = 0b100
B_3 = 0b10
B_2 = 0b1

def B_str(bm):
  if bm & B_OVER_5: return ">5"
  elif bm & B_5: return "=5"
  elif bm & B_OPEN_4: return "+4"
  elif bm & B_4: return "4"
  elif bm & B_OPEN_3: return "+3"
  elif bm & B_3: return "3"
  elif bm & B_2: return "2"
  return "."

class Analysis:
  """ Omok Anlyzer

  It takes a game board Mock5 and anlayze the state:
  - (TODO) (semi-)opend 4 connections
  - (TODO) 3-3, 3-4, 4-4 after one move
  """
  def __init__(self, game):
    self.game = game
    self.sz = game.height * game.width
    self.result = [None,]
    for i in range(2):
      a = [[0] * self.sz for i in range(4)]
      self.result.append(a)
    self.run_analysis()

  def print_result(self, c, dir):
    r = "====================================="
    r += "\n Analaysis, Color {}, Dir {}".format(c, dir)
    r += "\n   |"
    for i in range(self.game.width): r += " {:2d}".format(i)
    r += "\n--+"
    for i in range(self.game.width): r += "--"
    for i in range(self.game.height):
      r += "\n{:2d} |".format(i)
      for j in range(self.game.width):
        if self.game[i, j] > 0:
          r += " {:>2}".format(
              'o' if self.game[i, j] == c else 'X')
        else:
          r += " {:>2}".format(
              B_str(self.result[c][dir][i * self.game.width + j]))
    print(r)

  def is_marked(self, color, dir, idx, bm):
    return 0 != (self.result[color][dir][idx] & bm)

  def mark(self, color, dir, idx, bm):
    self.result[color][dir][idx] |= bm

  class _Window:
    def __init__(self, an, dir):
      self.board = [3] * 7
      self.dir = dir
      self.idx = [None] * 7
      self.n_color = [0, 0, 0, 5]
      self.p = 0
      self.an = an

    def color_at(self, off):
      return self.board[(self.p + off) % 7]

    def push(self, color=3, idx=None):
      self.idx[self.p] = idx
      self.board[self.p] = color
      self.p = (self.p + 1) % 7
      self.n_color[self.color_at(0)] -= 1
      self.n_color[self.color_at(5)] += 1

    def mark(self, c, off, bm):
      return self.an.mark(c, self.dir, self.idx[(self.p + off) % 7], bm)

    def _check_5(self, c):
      # Find empty idx
      j = 1
      while self.color_at(j) != 0: j += 1
      # If one of boundary has same color,
      # it is >=6-connect
      if self.color_at(0) == c or self.color_at(6) == c:
        self.mark(c, j, B_OVER_5)
      else:
        self.mark(c, j, B_5)

    def _check_4(self, c):
      for i in range(1, 6): self.mark(c, i, B_4)
      if self.color_at(1) == 0 and self.color_at(6) == 0:
        for i in range(2, 6):
          self.mark(c, i, B_OPEN_4)
      if self.color_at(0) == 0 and self.color_at(5) == 0:
        for i in range(1, 5):
          self.mark(c, i, B_OPEN_4)

    def _check_3(self, c):
      for i in range(1, 6): self.mark(c, i, B_3)
      if self.color_at(1) != 0 or self.color_at(5) != 0: return
      if self.color_at(2) == 0:
        if self.color_at(0) == 0:
          self.mark(c, 1, B_OPEN_3)
          self.mark(c, 2, B_OPEN_3)
        elif self.color_at(6) == 0:
          self.mark(c, 2, B_OPEN_3)
      elif self.color_at(4) == 0:
        if self.color_at(6) == 0:
          self.mark(c, 4, B_OPEN_3)
          self.mark(c, 5, B_OPEN_3)
        elif self.color_at(0) == 0:
          self.mark(c, 4, B_OPEN_3)
      else:
        if self.color_at(0) == 0:
          self.mark(c, 1, B_OPEN_3)
          self.mark(c, 3, B_OPEN_3)
        elif self.color_at(6) == 0:
          self.mark(c, 3, B_OPEN_3)
          self.mark(c, 5, B_OPEN_3)

    def _check_2(self, c):
      j = 1
      while self.color_at(j) == 0: j += 1
      for i in range(max(1, j - 2), min(j + 2, 5) + 1):
        self.mark(c, i, B_2)

    def check_connection(self):
      if self.n_color[3] == 0:
        c = 0
        if self.n_color[1] == 0: c = 2
        elif self.n_color[2] == 0: c = 1
        if c > 0:
          if self.n_color[c] == 4: self._check_5(c)
          elif self.n_color[c] == 3: self._check_4(c)
          elif self.n_color[c] == 2: self._check_3(c)
          elif self.n_color[c] == 1: self._check_2(c)

    def push_and_check(self, color=3, idx=None):
      self.push(color, idx)
      self.check_connection()

  def fill_result(self):
    # Row
    for i in range(self.game.height):
      w = self._Window(self, 0)
      for (r, c) in self.game.iter_row(i):
        w.push_and_check(self.game[r, c], self.game._reduce_index(r, c))
      w.push_and_check()
    # Col
    for i in range(self.game.width):
      w = self._Window(self, 1)
      for (r, c) in self.game.iter_column(i):
        w.push_and_check(self.game[r, c], self.game._reduce_index(r, c))
      w.push_and_check()
    # Diagonal
    for i in range(self.game.width + self.game.height - 1):
      w = self._Window(self, 2)
      for (r, c) in self.game.iter_right_down(i):
        w.push_and_check(self.game[r, c], self.game._reduce_index(r, c))
      w.push_and_check()
      w = self._Window(self, 3)
      for (r, c) in self.game.iter_left_down(i):
        w.push_and_check(self.game[r, c], self.game._reduce_index(r, c))
      w.push_and_check()

  def get_critical_at(self, color, dir, idx):
    bm = self.result[color][dir][idx]
    if bm & B_OVER_5: return N_OVER_5
    elif bm & B_5: return N_5
    elif bm & B_OPEN_4: return N_OPEN_4
    elif bm & B_4: return N_4
    elif bm & B_OPEN_3: return N_OPEN_3
    elif bm & B_3: return N_3
    elif bm & B_2: return N_2
    return 0

  def run_analysis(self):
    self.fill_result()

아래에서는 위에서 분석한 판의 가치를 바탕으로
가장 높은 가치의 수에 돌을 둡니다.

`lumiknit/mock5.py/mock5/agent_analysis_based.py`

In [ ]:
def agent_analysis_based(game):
  import math
  import random

  a = Analysis(game)

  my = game.player
  op = 3 - game.player

  max_s = -float('inf')
  max_i = None

  score = [0] * (game.height * game.width)
  for i in range(game.height * game.width):
    r, c = game._expand_index(i)
    if game[r, c] != 0: continue
    dr = r - game.height / 2
    dc = c - game.width / 2
    # Center is preffered
    score[i] -= math.sqrt((dr * dr) + (dc * dc))
    # Make some noise for random choice
    score[i] += random.random()
    for dir in range(4):
      m = a.get_critical_at(my, dir, i)
      o = a.get_critical_at(op, dir, i)
      score[i] += (10 ** m) + (10 ** o) * 0.7
    if score[i] > max_s:
      max_s = score[i]
      max_i = i
  if max_i is None: return None
  return game._expand_index(max_i)

이 agent로 게임을 실행하게 되면 다음과 같은 기보를 얻을 수 있습니다.

In [ ]:
Mock5().play(agent_analysis_based, agent_random)

In [ ]:
Mock5().play(agent_analysis_based, agent_analysis_based)

 [ Turn   0 ; 1P's turn (tone = O) ]
  | 0 1 2 3 4 5 6 7 8 9 A B C D E
--+------------------------------
0 | . . . . . . . . . . . . . . .
1 | . . . . . . . . . . . . . . .
2 | . . . . . . . . . . . . . . .
3 | . . . . . . . . . . . . . . .
4 | . . . . . . . . . . . . . . .
5 | . . . . . . . . . . . . . . .
6 | . . . . . . . . . . . . . . .
7 | . . . . . . . . . . . . . . .
8 | . . . . . . . . . . . . . . .
9 | . . . . . . . . . . . . . . .
A | . . . . . . . . . . . . . . .
B | . . . . . . . . . . . . . . .
C | . . . . . . . . . . . . . . .
D | . . . . . . . . . . . . . . .
E | . . . . . . . . . . . . . . .
 [ Turn   1 ; 2P's turn (tone = X) ]
  | 0 1 2 3 4 5 6 7 8 9 A B C D E
--+------------------------------
0 | . . . . . . . . . . . . . . .
1 | . . . . . . . . . . . . . . .
2 | . . . . . . . . . . . . . . .
3 | . . . . . . . . . . . . . . .
4 | . . . . . . . . . . . . . . .
5 | . . . . . . . . . . . . . . .
6 | . . . . . . . . . . . . . . .
7 | . . . . . . . O . . . . . . .
8 | . . 

1

Agent를 실행하다 보면 위 agent가 몇가지 특징을
알 수 있습니다.

- 한 수로 게임을 끝낼 수 있다면 그 수를
두지만, 몇 수에 걸쳐서 해당 형태가 나오게 되는 경우는
찾아내지 못합니다.
- 돌이 얼마나 연결되었냐에 따라
기계적으로 가치를 판단하다보니, 굉장히 방어적인 수를
둡니다.
공격적으로 돌을 이어서 공간을 확보하는 대신에,
상대의 수가 조금이라도 위협적으로 보이면 (=
돌이 어느 정도 이어지면) 바로바로 막아섭니다.